# Item: Neural Networks in PyTorch

#### References

1. **Neural Networks:** https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
2. **```torch.nn```:** https://pytorch.org/docs/stable/nn.html
3. **```torch.optim```:** https://pytorch.org/docs/stable/optim.html
4. **Mean squared error:** https://en.wikipedia.org/wiki/Mean_squared_error

## Neural Networks in PyTorch

Neural networks can be constructed using the ```torch.nn``` package. The ```torch.nn``` package depends on ```autograd``` package. An ```nn.Module``` contains layers, and a method ```forward(input)``` that returns the output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: ```weight = weight - learning_rate * gradient```


---

**Remark**

```torch.nn``` only supports mini-batches. The entire ```torch.nn`` package only supports inputs that are a mini-batch of samples, and not a single sample. For example, ```nn.Conv2d``` will take in a 4D Tensor of ```nSamples x nChannels x Height x Width```.

If you have a single sample, just use ```input.unsqueeze(0)``` to add a fake batch dimension.

---

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [13]:
classifier = ImageClassifier()
print(classifier)

ImageClassifier(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


---

**Remark**

You just have to define the ```forward``` function, and the ```backward``` function (where gradients are computed) is automatically defined for you using ```autograd```. You can use any of the Tensor operations in the ```forward``` function.

---

The learnable parameters of a model are returned by the ```parameters()``` function.

In [14]:
params = list(classifier.parameters())
print(params[0].size())  # conv1's .weight

torch.Size([6, 1, 5, 5])


### Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target. There are several different loss functions under the ```torch.nn``` package . A simple loss is: ```nn.MSELoss``` which computes the mean-squared error between the input and the target.

In [9]:
output = classifier(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not method

### Backpropagation

To backpropagate the error all we have to do is to ```loss.backward()```. 

---

**Remark**

You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

---

Now we shall call ```loss.backward()```, and have a look at ```conv1```’s bias gradients before and after the backward.

In [10]:
classifier.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(classifier.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(classifier.conv1.bias.grad)

conv1.bias.grad before backward
None


NameError: name 'loss' is not defined

### Updating the weights of the network

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD): 

```weight = weight - learning_rate * gradient```

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: ```torch.optim``` that implements all these methods. Using it is very simple:

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(classifier.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = classifier(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update